**извлечение признаков из текста на естественном языке**

очистка текста и токенизация

_Евгений Борисов <esborisov@sevsu.ru>_

---

In [2]:
from platform import python_version
display(python_version())

'3.11.8'

In [3]:
import re
import numpy as np
import numpy.random as rng
import pandas as pd
# from tqdm import tqdm
#from tqdm.notebook import tqdm

np.set_printoptions(precision=2) # вывод на печать чисел до 2 знака
pd.options.display.max_colwidth = 200 

#tqdm.pandas()

---

Проект Natasha. Набор качественных открытых инструментов для обработки естественного русского языка (NLP).   
https://habr.com/ru/post/516098/

Corus — коллекция русскоязычных NLP-датасетов.   
https://natasha.github.io/corus/

## текст

In [4]:
# # датасеты для примеров  https://disk.yandex.ru/d/3_WAa7SgrQYBzw
# # загружаем текст
# import gzip
# with gzip.open('../data/dostoevsky-besy-p2.txt.gz','rt',encoding='utf-8') as f: text = f.read()     
# # with gzip.open('../data/lobas-taxisty.txt.gz','rt',encoding='utf-8') as f: data = f.read()     
# display(len(text))
# print(text[85:1000])

In [5]:
# Ф.М.Достоевский "Бесы"
text = '''Прошло восемь дней. Теперь, когда уже все прошло, и я пишу хронику, мы уже знаем в
чем дело; но тогда мы еще ничего не знали, и естественно, что нам представлялись
странными разные вещи. По крайней мере мы со Степаном Трофимовичем в первое
время заперлись и с испугом наблюдали издали. Я-то кой-куда еще выходил и
по-прежнему приносил ему разные вести, без чего он и пробыть не мог.

Нечего и говорить, что по городу пошли самые разнообразные слухи, то-есть
насчет пощечины, обморока Лизаветы Николаевны и прочего случившегося в то
воскресенье. Но удивительно нам было то: через кого это все могло так скоро и
точно выйти наружу? Ни одно из присутствовавших тогда лиц не имело бы, кажется,
ни нужды, ни выгоды нарушить секрет происшедшего. Прислуги тогда не было; один
Лебядкин мог бы что-нибудь разболтать, не столько по злобе, потому что вышел
тогда в крайнем испуге (а страх к врагу уничтожает и злобу к нему)'''

## простой токенайзер

In [6]:
# пробел - разделитель
text.split()

['Прошло',
 'восемь',
 'дней.',
 'Теперь,',
 'когда',
 'уже',
 'все',
 'прошло,',
 'и',
 'я',
 'пишу',
 'хронику,',
 'мы',
 'уже',
 'знаем',
 'в',
 'чем',
 'дело;',
 'но',
 'тогда',
 'мы',
 'еще',
 'ничего',
 'не',
 'знали,',
 'и',
 'естественно,',
 'что',
 'нам',
 'представлялись',
 'странными',
 'разные',
 'вещи.',
 'По',
 'крайней',
 'мере',
 'мы',
 'со',
 'Степаном',
 'Трофимовичем',
 'в',
 'первое',
 'время',
 'заперлись',
 'и',
 'с',
 'испугом',
 'наблюдали',
 'издали.',
 'Я-то',
 'кой-куда',
 'еще',
 'выходил',
 'и',
 'по-прежнему',
 'приносил',
 'ему',
 'разные',
 'вести,',
 'без',
 'чего',
 'он',
 'и',
 'пробыть',
 'не',
 'мог.',
 'Нечего',
 'и',
 'говорить,',
 'что',
 'по',
 'городу',
 'пошли',
 'самые',
 'разнообразные',
 'слухи,',
 'то-есть',
 'насчет',
 'пощечины,',
 'обморока',
 'Лизаветы',
 'Николаевны',
 'и',
 'прочего',
 'случившегося',
 'в',
 'то',
 'воскресенье.',
 'Но',
 'удивительно',
 'нам',
 'было',
 'то:',
 'через',
 'кого',
 'это',
 'все',
 'могло',
 'так',
 'с

__*такой метод режет строку только на слова, предложения не разделяет*__

In [7]:
# собираем словарь
words = sorted( set( text.split()) )
display(len(words))
words

121

['(а',
 'Лебядкин',
 'Лизаветы',
 'Нечего',
 'Ни',
 'Николаевны',
 'Но',
 'По',
 'Прислуги',
 'Прошло',
 'Степаном',
 'Теперь,',
 'Трофимовичем',
 'Я-то',
 'без',
 'бы',
 'бы,',
 'было',
 'было;',
 'в',
 'вести,',
 'вещи.',
 'восемь',
 'воскресенье.',
 'врагу',
 'время',
 'все',
 'выгоды',
 'выйти',
 'выходил',
 'вышел',
 'говорить,',
 'городу',
 'дело;',
 'дней.',
 'ему',
 'естественно,',
 'еще',
 'заперлись',
 'злобе,',
 'злобу',
 'знаем',
 'знали,',
 'и',
 'из',
 'издали.',
 'имело',
 'испуге',
 'испугом',
 'к',
 'кажется,',
 'когда',
 'кого',
 'кой-куда',
 'крайней',
 'крайнем',
 'лиц',
 'мере',
 'мог',
 'мог.',
 'могло',
 'мы',
 'наблюдали',
 'нам',
 'наружу?',
 'нарушить',
 'насчет',
 'не',
 'нему)',
 'ни',
 'ничего',
 'но',
 'нужды,',
 'обморока',
 'один',
 'одно',
 'он',
 'первое',
 'пишу',
 'по',
 'по-прежнему',
 'потому',
 'пошли',
 'пощечины,',
 'представлялись',
 'приносил',
 'присутствовавших',
 'пробыть',
 'происшедшего.',
 'прочего',
 'прошло,',
 'разболтать,',
 'разнооб

In [8]:
# text

## NLTK токенайзер

In [9]:
# NLTK package manager
import nltk

nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [10]:
# NLTK токенайзер
from nltk.tokenize import sent_tokenize as nltk_sentence_split
from nltk.tokenize import word_tokenize as nltk_tokenize_word

text_ = [ 
    nltk_tokenize_word(s,language='russian') # разбиваем предложения на слова
    for s in nltk_sentence_split(text,language='russian') # режем текст на отдельные предложения
]

display( len(text_) )
display( text_ )

8

[['Прошло', 'восемь', 'дней', '.'],
 ['Теперь',
  ',',
  'когда',
  'уже',
  'все',
  'прошло',
  ',',
  'и',
  'я',
  'пишу',
  'хронику',
  ',',
  'мы',
  'уже',
  'знаем',
  'в',
  'чем',
  'дело',
  ';',
  'но',
  'тогда',
  'мы',
  'еще',
  'ничего',
  'не',
  'знали',
  ',',
  'и',
  'естественно',
  ',',
  'что',
  'нам',
  'представлялись',
  'странными',
  'разные',
  'вещи',
  '.'],
 ['По',
  'крайней',
  'мере',
  'мы',
  'со',
  'Степаном',
  'Трофимовичем',
  'в',
  'первое',
  'время',
  'заперлись',
  'и',
  'с',
  'испугом',
  'наблюдали',
  'издали',
  '.'],
 ['Я-то',
  'кой-куда',
  'еще',
  'выходил',
  'и',
  'по-прежнему',
  'приносил',
  'ему',
  'разные',
  'вести',
  ',',
  'без',
  'чего',
  'он',
  'и',
  'пробыть',
  'не',
  'мог',
  '.'],
 ['Нечего',
  'и',
  'говорить',
  ',',
  'что',
  'по',
  'городу',
  'пошли',
  'самые',
  'разнообразные',
  'слухи',
  ',',
  'то-есть',
  'насчет',
  'пощечины',
  ',',
  'обморока',
  'Лизаветы',
  'Николаевны',
  'и'

In [11]:
# собираем словарь
words = sorted( set.union( *[ set(s) for s in text_ ] ) )
display(len(words))
display( words )

124

['(',
 ')',
 ',',
 '.',
 ':',
 ';',
 '?',
 'Лебядкин',
 'Лизаветы',
 'Нечего',
 'Ни',
 'Николаевны',
 'Но',
 'По',
 'Прислуги',
 'Прошло',
 'Степаном',
 'Теперь',
 'Трофимовичем',
 'Я-то',
 'а',
 'без',
 'бы',
 'было',
 'в',
 'вести',
 'вещи',
 'восемь',
 'воскресенье',
 'врагу',
 'время',
 'все',
 'выгоды',
 'выйти',
 'выходил',
 'вышел',
 'говорить',
 'городу',
 'дело',
 'дней',
 'ему',
 'естественно',
 'еще',
 'заперлись',
 'злобе',
 'злобу',
 'знаем',
 'знали',
 'и',
 'из',
 'издали',
 'имело',
 'испуге',
 'испугом',
 'к',
 'кажется',
 'когда',
 'кого',
 'кой-куда',
 'крайней',
 'крайнем',
 'лиц',
 'мере',
 'мог',
 'могло',
 'мы',
 'наблюдали',
 'нам',
 'наружу',
 'нарушить',
 'насчет',
 'не',
 'нему',
 'ни',
 'ничего',
 'но',
 'нужды',
 'обморока',
 'один',
 'одно',
 'он',
 'первое',
 'пишу',
 'по',
 'по-прежнему',
 'потому',
 'пошли',
 'пощечины',
 'представлялись',
 'приносил',
 'присутствовавших',
 'пробыть',
 'происшедшего',
 'прочего',
 'прошло',
 'разболтать',
 'разнообразны

In [ ]:
# text

NLTK обеспечивает разделение на отдельные предложения   
и более качественную токенизацию   
разделяя конструкции типа '(Послышался'   

## токенайзер для русского языка из пакета  Natasha 

In [12]:
# !pip install natasha

In [13]:
from razdel import sentenize
from razdel import tokenize

text_ = [ [ t.text for t in tokenize(s.text) ] for s in sentenize(text) ]
display(text_)

[['Прошло', 'восемь', 'дней', '.'],
 ['Теперь',
  ',',
  'когда',
  'уже',
  'все',
  'прошло',
  ',',
  'и',
  'я',
  'пишу',
  'хронику',
  ',',
  'мы',
  'уже',
  'знаем',
  'в',
  'чем',
  'дело',
  ';',
  'но',
  'тогда',
  'мы',
  'еще',
  'ничего',
  'не',
  'знали',
  ',',
  'и',
  'естественно',
  ',',
  'что',
  'нам',
  'представлялись',
  'странными',
  'разные',
  'вещи',
  '.'],
 ['По',
  'крайней',
  'мере',
  'мы',
  'со',
  'Степаном',
  'Трофимовичем',
  'в',
  'первое',
  'время',
  'заперлись',
  'и',
  'с',
  'испугом',
  'наблюдали',
  'издали',
  '.'],
 ['Я-то',
  'кой-куда',
  'еще',
  'выходил',
  'и',
  'по-прежнему',
  'приносил',
  'ему',
  'разные',
  'вести',
  ',',
  'без',
  'чего',
  'он',
  'и',
  'пробыть',
  'не',
  'мог',
  '.'],
 ['Нечего',
  'и',
  'говорить',
  ',',
  'что',
  'по',
  'городу',
  'пошли',
  'самые',
  'разнообразные',
  'слухи',
  ',',
  'то-есть',
  'насчет',
  'пощечины',
  ',',
  'обморока',
  'Лизаветы',
  'Николаевны',
  'и'

In [14]:
# собираем словарь
words = sorted( set.union( *[ set(s) for s in text_ ] ) )
display(len(words))
display(words)

124

['(',
 ')',
 ',',
 '.',
 ':',
 ';',
 '?',
 'Лебядкин',
 'Лизаветы',
 'Нечего',
 'Ни',
 'Николаевны',
 'Но',
 'По',
 'Прислуги',
 'Прошло',
 'Степаном',
 'Теперь',
 'Трофимовичем',
 'Я-то',
 'а',
 'без',
 'бы',
 'было',
 'в',
 'вести',
 'вещи',
 'восемь',
 'воскресенье',
 'врагу',
 'время',
 'все',
 'выгоды',
 'выйти',
 'выходил',
 'вышел',
 'говорить',
 'городу',
 'дело',
 'дней',
 'ему',
 'естественно',
 'еще',
 'заперлись',
 'злобе',
 'злобу',
 'знаем',
 'знали',
 'и',
 'из',
 'издали',
 'имело',
 'испуге',
 'испугом',
 'к',
 'кажется',
 'когда',
 'кого',
 'кой-куда',
 'крайней',
 'крайнем',
 'лиц',
 'мере',
 'мог',
 'могло',
 'мы',
 'наблюдали',
 'нам',
 'наружу',
 'нарушить',
 'насчет',
 'не',
 'нему',
 'ни',
 'ничего',
 'но',
 'нужды',
 'обморока',
 'один',
 'одно',
 'он',
 'первое',
 'пишу',
 'по',
 'по-прежнему',
 'потому',
 'пошли',
 'пощечины',
 'представлялись',
 'приносил',
 'присутствовавших',
 'пробыть',
 'происшедшего',
 'прочего',
 'прошло',
 'разболтать',
 'разнообразны

## стеминг  для русского языка из пакета  NLTK

In [15]:
from nltk.tokenize import sent_tokenize as nltk_sentence_split
from nltk.tokenize import word_tokenize as nltk_tokenize_word

from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('russian')

text_ = [ 
    [ # разбиваем предложения на слова
        stemmer.stem(t) # выполняем стеминг
        for t in nltk_tokenize_word(s,language='russian') 
    ] # разбиваем предложения на слова
    for s in nltk_sentence_split(text,language='russian') # режем текст на отдельные предложения
]

display( len(text_) )
display(text_)

8

[['прошл', 'восем', 'дне', '.'],
 ['тепер',
  ',',
  'когд',
  'уж',
  'все',
  'прошл',
  ',',
  'и',
  'я',
  'пиш',
  'хроник',
  ',',
  'мы',
  'уж',
  'зна',
  'в',
  'чем',
  'дел',
  ';',
  'но',
  'тогд',
  'мы',
  'ещ',
  'нич',
  'не',
  'знал',
  ',',
  'и',
  'естествен',
  ',',
  'что',
  'нам',
  'представля',
  'стран',
  'разн',
  'вещ',
  '.'],
 ['по',
  'крайн',
  'мер',
  'мы',
  'со',
  'степан',
  'трофимович',
  'в',
  'перв',
  'врем',
  'заперл',
  'и',
  'с',
  'испуг',
  'наблюда',
  'изда',
  '.'],
 ['я-т',
  'кой-куд',
  'ещ',
  'выход',
  'и',
  'по-прежн',
  'принос',
  'ем',
  'разн',
  'вест',
  ',',
  'без',
  'чег',
  'он',
  'и',
  'проб',
  'не',
  'мог',
  '.'],
 ['неч',
  'и',
  'говор',
  ',',
  'что',
  'по',
  'город',
  'пошл',
  'сам',
  'разнообразн',
  'слух',
  ',',
  'то-ест',
  'насчет',
  'пощечин',
  ',',
  'обморок',
  'лизавет',
  'николаевн',
  'и',
  'проч',
  'случ',
  'в',
  'то',
  'воскресен',
  '.'],
 ['но',
  'удивительн',
  '

In [16]:
# собираем словарь
words = sorted( set.union( *[ set(s) for s in text_ ] ) )
display(len(words))
display(words)

117

['(',
 ')',
 ',',
 '.',
 ':',
 ';',
 '?',
 'а',
 'без',
 'бы',
 'был',
 'в',
 'вест',
 'вещ',
 'восем',
 'воскресен',
 'враг',
 'врем',
 'все',
 'выгод',
 'выйт',
 'выход',
 'вышел',
 'говор',
 'город',
 'дел',
 'дне',
 'ем',
 'естествен',
 'ещ',
 'заперл',
 'злоб',
 'зна',
 'знал',
 'и',
 'из',
 'изда',
 'имел',
 'испуг',
 'к',
 'кажет',
 'ког',
 'когд',
 'кой-куд',
 'крайн',
 'лебядкин',
 'лизавет',
 'лиц',
 'мер',
 'мог',
 'могл',
 'мы',
 'наблюда',
 'нам',
 'наруж',
 'наруш',
 'насчет',
 'не',
 'нем',
 'неч',
 'ни',
 'николаевн',
 'нич',
 'но',
 'нужд',
 'обморок',
 'один',
 'одн',
 'он',
 'перв',
 'пиш',
 'по',
 'по-прежн',
 'пот',
 'пошл',
 'пощечин',
 'представля',
 'принос',
 'прислуг',
 'присутствова',
 'проб',
 'происшедш',
 'проч',
 'прошл',
 'разболта',
 'разн',
 'разнообразн',
 'с',
 'сам',
 'секрет',
 'скор',
 'слух',
 'случ',
 'со',
 'степан',
 'стольк',
 'стран',
 'страх',
 'так',
 'тепер',
 'то',
 'то-ест',
 'тогд',
 'точн',
 'трофимович',
 'удивительн',
 'уж',
 'уничт

после применения стеминга размер словаря уменьшился

## лемматизатор для русского языка из пакета Natasha

In [17]:
# text

In [18]:
# !pip install pymorphy3
# !pip install pymorphy3-dicts-ru
# !pip install natasha

In [19]:
from natasha import Doc
from natasha import Segmenter
from natasha import MorphVocab
from natasha import NewsEmbedding
from natasha import NewsMorphTagger

In [20]:
doc = Doc(text) 
display(doc)

Doc(text='Прошло восемь дней. Теперь, когда уже все прошло,...)

In [21]:
doc.segment( Segmenter() ) # выполняем сегментацию строки на токены
n =  rng.randint(len(doc.sents))
display( doc.sents[n].tokens )

[DocToken(start=187, stop=189, text='По'),
 DocToken(start=190, stop=197, text='крайней'),
 DocToken(start=198, stop=202, text='мере'),
 DocToken(start=203, stop=205, text='мы'),
 DocToken(start=206, stop=208, text='со'),
 DocToken(start=209, stop=217, text='Степаном'),
 DocToken(start=218, stop=230, text='Трофимовичем'),
 DocToken(start=231, stop=232, text='в'),
 DocToken(start=233, stop=239, text='первое'),
 DocToken(start=240, stop=245, text='время'),
 DocToken(start=246, stop=255, text='заперлись'),
 DocToken(start=256, stop=257, text='и'),
 DocToken(start=258, stop=259, text='с'),
 DocToken(start=260, stop=267, text='испугом'),
 DocToken(start=268, stop=277, text='наблюдали'),
 DocToken(start=278, stop=284, text='издали'),
 DocToken(start=284, stop=285, text='.')]

In [22]:
doc.tag_morph( NewsMorphTagger( NewsEmbedding() ) ) 
# выполняем морфологический анализ
display(doc)

Doc(text='Прошло восемь дней. Теперь, когда уже все прошло,..., tokens=[...], sents=[...])

In [23]:
display( doc.sents[n].tokens )

[DocToken(start=187, stop=189, text='По', pos='ADP'),
 DocToken(start=190, stop=197, text='крайней', pos='ADJ', feats=<Dat,Pos,Fem,Sing>),
 DocToken(start=198, stop=202, text='мере', pos='NOUN', feats=<Inan,Dat,Fem,Sing>),
 DocToken(start=203, stop=205, text='мы', pos='PRON', feats=<Nom,Plur,1>),
 DocToken(start=206, stop=208, text='со', pos='ADP'),
 DocToken(start=209, stop=217, text='Степаном', pos='PROPN', feats=<Anim,Ins,Masc,Sing>),
 DocToken(start=218, stop=230, text='Трофимовичем', pos='PROPN', feats=<Anim,Ins,Masc,Sing>),
 DocToken(start=231, stop=232, text='в', pos='ADP'),
 DocToken(start=233, stop=239, text='первое', pos='ADJ', feats=<Inan,Acc,Pos,Neut,Sing>),
 DocToken(start=240, stop=245, text='время', pos='NOUN', feats=<Inan,Acc,Neut,Sing>),
 DocToken(start=246, stop=255, text='заперлись', pos='VERB', feats=<Perf,Ind,Plur,Past,Fin,Mid>),
 DocToken(start=256, stop=257, text='и', pos='PART'),
 DocToken(start=258, stop=259, text='с', pos='ADP'),
 DocToken(start=260, stop=267,

In [24]:
# !pip install --upgrade pymorphy2

In [25]:
# выполняем лематизацию
morph_vocab = MorphVocab() # лемматизатор

for t in doc.tokens: 
    t.lemmatize(morph_vocab)

In [26]:
# собираем нормализованный текст 
text_ =  [ [ t.lemma for t in s.tokens ] for s in doc.sents ]
display(text_)

[['пройти', 'восемь', 'день', '.'],
 ['теперь',
  ',',
  'когда',
  'уже',
  'весь',
  'пройти',
  ',',
  'и',
  'я',
  'писать',
  'хроника',
  ',',
  'мы',
  'уже',
  'знать',
  'в',
  'что',
  'дело',
  ';',
  'но',
  'тогда',
  'мы',
  'еще',
  'ничто',
  'не',
  'знать',
  ',',
  'и',
  'естественно',
  ',',
  'что',
  'мы',
  'представляться',
  'странный',
  'разный',
  'вещь',
  '.'],
 ['по',
  'крайний',
  'мера',
  'мы',
  'с',
  'степан',
  'трофимович',
  'в',
  'первый',
  'время',
  'запереться',
  'и',
  'с',
  'испуг',
  'наблюдать',
  'издать',
  '.'],
 ['я-то',
  'кой-куда',
  'еще',
  'выходить',
  'и',
  'по-прежнему',
  'приносить',
  'он',
  'разный',
  'весть',
  ',',
  'без',
  'что',
  'он',
  'и',
  'пробыть',
  'не',
  'мочь',
  '.'],
 ['нечего',
  'и',
  'говорить',
  ',',
  'что',
  'по',
  'город',
  'пойти',
  'самый',
  'разнообразный',
  'слух',
  ',',
  'то-есть',
  'насчет',
  'пощечина',
  ',',
  'обморок',
  'лизавета',
  'николаевич',
  'и',
  'про

In [27]:
# собираем словарь
words = sorted( set.union( *[ set(s) for s in text_ ] ) )
display(len(words))
display(words)

107

['(',
 ')',
 ',',
 '.',
 ':',
 ';',
 '?',
 'а',
 'без',
 'бы',
 'быть',
 'в',
 'весть',
 'весь',
 'вещь',
 'восемь',
 'воскресение',
 'враг',
 'время',
 'выгода',
 'выйти',
 'выходить',
 'говорить',
 'город',
 'дело',
 'день',
 'естественно',
 'еще',
 'запереться',
 'злоба',
 'знать',
 'и',
 'из',
 'издать',
 'иметь',
 'испуг',
 'к',
 'казаться',
 'когда',
 'кой-куда',
 'крайний',
 'кто',
 'лебядкин',
 'лизавета',
 'лицо',
 'мера',
 'мочь',
 'мы',
 'наблюдать',
 'наружу',
 'нарушить',
 'насчет',
 'не',
 'нечего',
 'ни',
 'николаевич',
 'ничто',
 'но',
 'нужда',
 'обморок',
 'один',
 'он',
 'первый',
 'писать',
 'по',
 'по-прежнему',
 'пойти',
 'потому',
 'пощечина',
 'представляться',
 'приносить',
 'прислуга',
 'присутствовать',
 'пробыть',
 'произойти',
 'пройти',
 'прочий',
 'разболтать',
 'разнообразный',
 'разный',
 'с',
 'самый',
 'секрет',
 'скоро',
 'слух',
 'случиться',
 'степан',
 'столько',
 'странный',
 'страх',
 'так',
 'теперь',
 'то-есть',
 'тогда',
 'тот',
 'точно',
 'т